In [1]:
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1011. Emission Mixtures/P1011.3. Analyses/P1011.3.1. Biomarker Distribution Analysis/Output')
cur_date = "050123"

library(readxl)
library(tidyverse)
library(reshape2)
library(stats)
library(multcomp)

# reading in files
proteomics_5_df = data.frame(read_excel("Input/Proteomics_Data_5ug_040423.xlsx", sheet = 2))
proteomics_25_df = data.frame(read_excel("Input/Proteomics_Data_25ug_040423.xlsx", sheet = 2))
demographics_df = data.frame(read_excel("Input/Subject_Info_031723.xlsx", sheet = 2))

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


Loading required package: mvtnorm

Loading required package: survival

Loading required package: TH.data

Loading required package: MASS


Attaching package: ‘MASS’


The following object is masked from ‘package:dplyr’:

    select



Attaching package: ‘TH.data’


The following object is masked from ‘package:MASS’:

    geyser




Testing for statistical differences by comparing a protein expression between burn conditions (control, smoldering or flaming). (The condensate and concentration would remain consistent). Protein expression (intensity) was pseudo log transformed to reduce skew.

A non-parametric test, a Quade's ANCOVA, will be performed to control for sex as a covariate.

In [2]:
head(proteomics_25_df)
head(demographics_df)

,Subject_No,Subject_ID,Protein_Accession,Gene_Name,Description,Condensate,Burn_Condition,Concentration,Intensity,Intensity_pslog2
,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,1,F_1,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,1488600000,30.47131
2,2,M_2,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,3806600000,31.82586
3,3,M_3,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,3900000000,31.86083
4,4,F_4,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,5373200000,32.32313
5,5,F_5,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,2595200000,31.27320
6,6,M_6,CO3_HUMAN,C3,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,5780700000,32.42860


,Original_Subject_ID,Subject_ID,Subject_No,Sex,Age,Race,Ethnicity
,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>
1,68J,F_1,1,F,18,B,NH
2,53K,M_2,2,M,32,B,NH
3,21O,M_3,3,M,34,W,H
4,63O,F_4,4,F,32,W,H
5,67L,F_5,5,F,17,W,H
6,57N,M_6,6,M,59,W,NH


In [7]:
# combining data, but first making sex a factor
demographics_df = demographics_df %>%
    mutate(Sex = relevel(factor(ifelse(Sex == "M", 1, 0)), ref = "0"))

proteomics_25_sex_df = inner_join(proteomics_25_df[,-c(4:5)], demographics_df[,c(2,4)]) %>%
    # scaling the concentration values by protein normalizes the distribution
    group_by(Protein_Accession) %>%
    mutate(Scaled_Intensity = scale(Intensity_pslog2))
proteomics_5_sex_df = inner_join(proteomics_5_df[,-c(4:5)], demographics_df[,c(2,4)]) %>%
    # scaling the concentration values by protein normalizes the distribution
    group_by(Protein_Accession) %>%
    mutate(Scaled_Intensity = scale(Intensity_pslog2))

# putting burn condition into a factor
proteomics_25_sex_df$Burn_Condition = factor(proteomics_25_sex_df$Burn_Condition, levels = c("PBS", "S", "F"))
proteomics_5_sex_df$Burn_Condition = factor(proteomics_5_sex_df$Burn_Condition, levels = c("PBS", "S", "F"))

head(proteomics_25_sex_df)

Joining, by = "Subject_ID"
Joining, by = "Subject_ID"


Subject_No,Subject_ID,Protein_Accession,Condensate,Burn_Condition,Concentration,Intensity,Intensity_pslog2,Sex,Scaled_Intensity
<dbl>,<chr>,<chr>,<chr>,<fct>,<dbl>,<dbl>,<dbl>,<fct>,"<dbl[,1]>"
1,F_1,CO3_HUMAN,C,F,25,1488600000,30.47131,0,-1.29273172
2,M_2,CO3_HUMAN,C,F,25,3806600000,31.82586,0,-0.04856119
3,M_3,CO3_HUMAN,C,F,25,3900000000,31.86083,0,-0.01643970
4,F_4,CO3_HUMAN,C,F,25,5373200000,32.32313,0,0.40819624
5,F_5,CO3_HUMAN,C,F,25,2595200000,31.27320,0,-0.55618474
6,M_6,CO3_HUMAN,C,F,25,5780700000,32.42860,0,0.50506520


In [8]:
# contrasts show what compartments are being compared in anova
# these comparisons aren't what we want so I changed them in the function below
contrasts(proteomics_25_sex_df$Burn_Condition)

,S,F
PBS,0,0
S,1,0
F,0,1


               Table of Contrasts 
>               PBS |  S  |  F  |  Sum
>  - Contrast 1 |  -1  |  1  |  0  |   0
>  - Contrast 2 |  -1  |  0  |  1  |   0

> - Contrast 1: compares PBS to smoldering
> - Contrast 2: compares PBS to flaming

In [60]:
get_anova = function(df, conc){
    # first filtering the df and iterating through the condensate
    condensates = c("C", "P")
    tukey_df = data.frame()
    for (i in 1:length(condensates)){
        filtered_df = df %>%
            filter(Condensate == c(condensates[i], "PBS"))
        
        contrasts(filtered_df$Burn_Condition) = cbind(c(-1,1,0), c(-1,0,1)) # meaning is specified above
        anova = aov(Scaled_Intensity ~ Burn_Condition, data = filtered_df)
        
        # Tukey's post hoc test
        tukeys_anova = summary(glht(anova, linfct = mcp(Burn_Condition = "Tukey")), test = adjusted("none"))
        tukeys_test = tukeys_anova$test
        stat_pvalue = data.frame(tukeys_test$tstat, tukeys_test$pvalues) %>%
             rownames_to_column(var = "Comparison")
        
        # adding condensate and conc cols
        stat_pvalue$Condensate = condensates[i]
        stat_pvalue$Concentration = conc
        
        # adjusting p values
        stat_pvalue$`P Adj` = p.adjust(as.numeric(as.character(stat_pvalue$pvalues)), method = "fdr")
        #print(p.adjust(stat_pvalue$pvalues), method = "fdr")
        
        # saving values
        tukey_df = rbind(tukey_df, stat_pvalue)

    }
    
    # changing col names
    colnames(tukey_df)[2:3] = c("Statistic", "P Value")
     return(tukey_df)
}

# calling fn
proteomics_25_crude_anova = get_anova(proteomics_25_sex_df, 25)
proteomics_5_crude_anova = get_anova(proteomics_5_sex_df, 5)

ERROR: Error in `$<-.data.frame`(`*tmp*`, "P Adj", value = numeric(0)): replacement has 0 rows, data has 3


In [59]:
p.adjust(as.numeric(as.character(crude_anova_df$`P Value`)), method = "fdr")

[1] 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000
 [6] 0.0001452651 0.0000000000 0.0000000000 0.0000000000 0.1251418165
[11] 0.0000000000 0.0000000000

In [58]:
crude_anova_df = rbind(proteomics_25_crude_anova, proteomics_5_crude_anova)
crude_anova_df

Comparison,Statistic,P Value,Condensate,Concentration
<chr>,<dbl>,<dbl>,<chr>,<dbl>
S - PBS,25.167067,0.0000000000,C,25
F - PBS,35.387233,0.0000000000,C,25
F - S,10.220166,0.0000000000,C,25
S - PBS,47.826576,0.0000000000,P,25
F - PBS,44.005223,0.0000000000,P,25
F - S,-3.821353,0.0001331597,P,25
S - PBS,19.941011,0.0000000000,C,5
F - PBS,47.237998,0.0000000000,C,5
F - S,27.296987,0.0000000000,C,5


In [5]:
quade_test_v1 = function(df){
    # """
    # Running quade test after filtering for a protein and condensate using a loop and controlling for sex. 
    # Ultimately using this test to compare protein expression (control vs. flaming burn condition).

    # :param: protein dataframe
    # :output: a dataframe containing the protein, condensate, comparison, stat, p value, p adj

    # """
    
    # variables that will be iterated through
    proteins = unique(df$Protein_Accession)
    condensates = c("C", "P")
    
    values_df = data.frame()
     # iterating through each protein, condensate, and burn condition
    for(i in 1:length(proteins)){
        for(j in 1:length(condensates)){
                
            # control df
            control_df = df %>%
                filter(Protein_Accession == proteins[i], Condensate == "PBS")

            # flaming df
            flaming_df = df %>%
                filter(Protein_Accession == proteins[i], Condensate == condensates[j], Burn_Condition == "F")

            # quade's test
            quade_test = quade.test(Intensity_pslog2 ~ )

            # contains gene, condensate, burn conditions, conc, stat, and p value
            values_vector = cbind(proteins[i], condensates[j], "Control vs. Flaming",
                                  unique(df
statistic, quade_test$p.value)
            values_df = rbind(values_df, values_vector)
        }
    }

    
    # adding col names
    colnames(values_df) = c("Protein", "Condensate", "Comparison", "Concentration", "Statistic", "P Value")
    
    # calculating padj values
    values_df$`P Adj` = p.adjust(as.numeric(as.character(values_df$`P Value`)), method = "fdr")

    
    return(values_df)
}

ERROR: Error in parse(text = x, srcfile = src): <text>:29:56: unexpected ')'
28:             # quade's test
29:             quade_test = quade.test(Intensity_pslog2 ~ )
                                                           ^
